In [ ]:
import os
import sys
import time as t_util
import numpy as np
import pandas as pd
import yaml
import cftime
import xarray as xr


## Define folders

In [ ]:
#Read main paths
with open('../path_main.txt', 'r') as file:   path_main  = file.read()
with open('../path_CMIP6.txt', 'r') as file:  path_cmip6 = file.read()
    
dir_CMIP6   = path_cmip6
dir_scripts = f'{path_main}Scripts/'
dir_names   = f'{path_main}Scripts/Model_lists/'
dir_out1    = f'{path_main}Data/CMIP6/EMT/'
dir_out2    = f'{path_main}Data/CMIP6/GMT/'
if not os.path.exists(dir_out1):  os.mkdir(dir_out1)
if not os.path.exists(dir_out2):  os.mkdir(dir_out2)


## Prepare variables and parameters

In [ ]:
#Define limits for European box
lon_limits_EUR = [-10, 35]
lat_limits_EUR = [30, 70]

#Define scenarios
scenarios = ['historical', 'ssp585']

#Define models and SSPs which should be used
all_models = dict()
all_models['ssp585'] = []
with open(dir_names + 'Models_CMIP6_SSP585.txt', 'r') as filehandle:
    for line in filehandle:
        all_models['ssp585'].append(line[:-1])


## Calculation

In [ ]:
#Loop over models
for model in all_models['ssp585']:
    
    print(model)
    
    #Select ensemble member
    if model in ['CNRM-CM6-1', 'CNRM-ESM2-1', 'CNRM-CM6-1-HR', 'UKESM1-0-LL', 'MIROC-ES2L']:
        member = "r1i1p1f2"
    elif model in ['HadGEM3-GC31-LL', 'HadGEM3-GC31-MM']:
        member = "r1i1p1f3"
    else:
        member = "r1i1p1f1"            

    #Loop over scenarios
    for scen in scenarios:
        
        #Get file names
        dir_data = dir_CMIP6 + scen + '/tas/'
        fnames = [dir_data + file for file in os.listdir(dir_data) if model + '_' in file and member in file and 'tas_' in file and scen in file and '_Amon_' in file]
        if model=='MPI-ESM1-2-LR' and scen=='ssp585':  fnames = [file for file in fnames if '_gn_' in file]
        if len(fnames)!=1: sys.exit('Filename not uniquely defined')

        #Read data
        data = xr.open_dataset(fnames[0], use_cftime=True)

        #Convert longitude from [0, 360] to [-180, 180]
        data['lon'] = data['lon'].where(data['lon']<180, ((data['lon'] + 180) % 360) - 180)
        data = data.sortby('lon')
        
        #Select time period
        if scen=='historical':
            data = data.sel(time=slice('1950', '2014'))
        elif scen=='ssp585':
            data = data.sel(time=slice('2015', '2100'))
            
        #Calculate yearly mean
        ds_grid = data.resample(time='1Y').mean('time')

        #Get mask for Europe
        mask_EUR = ((data['lon']>=lon_limits_EUR[0]) & (data['lon']<=lon_limits_EUR[1]) & 
                    (data['lat']>=lat_limits_EUR[0]) & (data['lat']<=lat_limits_EUR[1]))
        
        #Mask Europe (only for EMT!)
        ds_masked = ds_grid.where(mask_EUR)
        
        #Calculate EMT
        area_weights = np.cos(ds_masked['lat'] * np.pi/180)
        ds_EMT = ds_masked.tas.weighted(area_weights).mean(('lat', 'lon'))  
        ds_EMT = ds_EMT.to_dataset(name='tas')        
        
        #Calculate GMT
        area_weights = np.cos(ds_grid['lat'] * np.pi/180)
        ds_GMT = ds_grid.tas.weighted(area_weights).mean(('lat', 'lon'))  
        ds_GMT = ds_GMT.to_dataset(name='tas')        
        
        #Create output file names
        time_str  = str(data.time[0].dt.year.values) + '-' + str(data.time[-1].dt.year.values)
        fname_out_EMT = dir_out1 + 'EMT_year_' + scen + '_' + model + '_' + time_str + ".nc"
        fname_out_GMT = dir_out2 + 'GMT_year_' + scen + '_' + model + '_' + time_str + ".nc"

        #Save in file
        if os.path.exists(fname_out_EMT): os.remove(fname_out_EMT)
        if os.path.exists(fname_out_tas): os.remove(fname_out_tas)
        ds_EMT.to_netcdf(fname_out_EMT)
        ds_grid.to_netcdf(fname_out_tas)


## Calculate GMT for full historical period

In [ ]:
#Loop over models
for model in all_models['ssp585']:
    
    print(model)
    
    #Select ensemble member
    if model in ['CNRM-CM6-1', 'CNRM-ESM2-1', 'CNRM-CM6-1-HR', 'UKESM1-0-LL', 'MIROC-ES2L']:
        member = "r1i1p1f2"
    elif model in ['HadGEM3-GC31-LL', 'HadGEM3-GC31-MM']:
        member = "r1i1p1f3"
    else:
        member = "r1i1p1f1"            

    #Get file names
    dir_data = dir_CMIP6 + 'historical/tas/'
    fnames = [dir_data + file for file in os.listdir(dir_data) if model + '_' in file and member in file and 'tas_' in file and 'historical' in file and '_Amon_' in file]
    if len(fnames)!=1: sys.exit('Filename not uniquely defined')

    #Read data
    data = xr.open_dataset(fnames[0], use_cftime=True)

    #Convert longitude from [0, 360] to [-180, 180]
    data['lon'] = data['lon'].where(data['lon']<180, ((data['lon'] + 180) % 360) - 180)
    data = data.sortby('lon')

    #Select time period
    data = data.sel(time=slice('1850', '2014'))

    #Calculate yearly mean
    ds_grid = data.resample(time='1Y').mean('time')

    #Calculate GMT
    area_weights = np.cos(ds_grid['lat'] * np.pi/180)
    ds_GMT = ds_grid.tas.weighted(area_weights).mean(('lat', 'lon'))  
    ds_GMT = ds_GMT.to_dataset(name='tas')        

    #Create output file names
    time_str  = str(data.time[0].dt.year.values) + '-' + str(data.time[-1].dt.year.values)
    fname_out_GMT = dir_out2 + 'GMT_year_historical_' + model + '_' + time_str + ".nc"

    #Save in file
    if os.path.exists(fname_out_GMT): os.remove(fname_out_GMT)
    ds_GMT.to_netcdf(fname_out_GMT)
